In [8]:
import plotly.express as px
import pandas as pd
import psycopg2

def fetch_data(query, db_config):
    """
    Fetch data from a PostgreSQL database.

    Parameters:
    - query: SQL query to fetch data.
    - db_config: A dictionary containing database configurations.

    Returns:
    - List of tuples containing the query results.
    """

    # Extract the database configuration details
    dbname = db_config['dbname']
    user = db_config['user']
    password = db_config['password']
    host = db_config['host']
    port = db_config['port']

    # Establish the connection
    try:
        conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
        cur = conn.cursor()

        # Execute the query
        cur.execute(query)
        
        # Extract column names from the cursor description
        column_names = [desc[0] for desc in cur.description]

        # Fetch results
        results = cur.fetchall()

        # Create a DataFrame with the results and column names
        df = pd.DataFrame(results, columns=column_names)

        # Clean up and return
        cur.close()
        conn.close()
        return df

    except psycopg2.Error as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

# Usage Example:
db_config = {
    'dbname': 'tutorialdb',
    'user': 'postgres',
    'password': 'P0stgres@2020',
    'host': 'localhost',
    'port': '5432'
}
query = "SELECT * FROM sample"

# Assuming you have a function to fetch data called fetch_data()
df = fetch_data(query, db_config)

print(df)
fig = px.bar(df, x='productname', y='quantity', title='Total Quantity of Each Product Sold')
fig.show()

category_data = df.groupby('category').sum()['quantity']
fig = px.pie(category_data, values='quantity', names=category_data.index, title='Sales Proportion by Category')
fig.show()

# Assuming your Date column is in string format. Convert it to datetime for better plotting.
df['Date'] = pd.to_datetime(df['date'])

sales_over_time = df.groupby('date').sum()['quantity']
fig = px.line(sales_over_time, x=sales_over_time.index, y='quantity', title='Sales Trend Over Time')
fig.show()

from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('visualization.html')

@app.route('/fetch_data', methods=['GET'])
def fetch_data_route():
    # Use the fetch_data function you already have to get the latest data.
    query = "SELECT * FROM sample"
    data = fetch_data(query, db_config)
    
    # Convert the data to JSON and return
    return jsonify(data.to_dict(orient='records'))

if __name__ == '__main__':
    app.run(debug=True)


       transactionid        date             time productname    category  \
0                  1  2023-01-01         00:00:00      Banana      Fruits   
1                  2  2023-01-01         00:05:00      Potato  Vegetables   
2                  3  2023-01-01         00:10:00      Tomato  Vegetables   
3                  4  2023-01-01         00:15:00      Potato  Vegetables   
4                  5  2023-01-01         00:20:00      Tomato  Vegetables   
...              ...         ...              ...         ...         ...   
22887          22888  2023-10-14  22:55:20.884039     Chicken       Meats   
22888          22889  2023-10-14  22:55:20.893015       Bread      Bakery   
22889          22890  2023-10-14  22:55:20.906980      Banana      Fruits   
22890          22891  2023-10-14  22:55:20.916950     Chicken       Meats   
22891          22892  2023-10-14  22:55:20.926925     Chicken       Meats   

       price  quantity  
0        0.2         1  
1        0.4        14  


<ipython-input-8-cac2be0650a8>:67: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



<ipython-input-8-cac2be0650a8>:74: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

